In [1]:
import os
import openai

openai.api_key = os.environ.get("OPENAI_API_KEY")
import nest_asyncio;
llama_key = "llx-ypESljhKJByisEBTaLPBRL97bsYJo1ySw8cbbbD3WDLI5Ji1"

nest_asyncio.apply()


In [2]:
from llama_index.readers.file import UnstructuredReader
from pathlib import Path
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_parse import LlamaParse

try:
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir="./storage")
    # load index
    index = load_index_from_storage(storage_context)
except:
    # parser = LlamaParse(api_key = llmama_api, result_type ="markdown")
    # file_extractor =  {".pdf": parser}
    # documents = SimpleDirectoryReader(
    #     "./data", file_extractor= file_extractor).load_data(show_progress=True)
    # print(documents)
    from llama_parse.base import ResultType, Language
    parser = LlamaParse(api_key= llama_key,result_type=ResultType.MD,language=Language.ENGLISH)
    documents = parser.load_data('D:\VSCode\VGU\chainlitGPT\data\Clean.Code.A.Handbook.of.Agile.Software.Craftsmanship.pdf')
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist()


Started parsing the file under job_id 396afc15-b898-417e-a636-a0c70e822b59


In [9]:
# Load indices from disk
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core import load_index_from_storage
loaded_storage_context = StorageContext.from_defaults(persist_dir="./storage")
loaded_index = load_index_from_storage(loaded_storage_context)

In [10]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_engine_tool = QueryEngineTool(
    query_engine=loaded_index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_index",
        description="Use this for answering queries on the Clean Code handbook or any other queries related to coding.",
    ),
)

In [11]:
from llama_index.llms.openai import OpenAI
from llama_index.core.query_engine import SubQuestionQueryEngine

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=[query_engine_tool],
    llm=OpenAI(model="gpt-4o-mini", temperature= 0.2),
)

In [44]:
# query_engine_tool = QueryEngineTool(
#     query_engine=query_engine,
#     metadata=ToolMetadata(
#         name="sub_question_query_engine",
#         description="useful for when you want to answer queries that require analyzing multiple SEC 10-K documents for Uber",
#     ),
# )

In [12]:
tools =[query_engine_tool]
tools

In [13]:
import pickle

# Save the `tools` variable to a .pkl file
with open('tools.pkl', 'wb') as file:
    pickle.dump(tools, file)

In [20]:
from llama_index.agent.openai import OpenAIAgent

agent = OpenAIAgent.from_tools(tools, verbose=True)

In [21]:
response = agent.chat(
    "Explain chapter 7 in the book"
)
print(str(response))

Added user message to memory: Explain chapter 7 in the book
=== Calling Function ===
Calling function: vector_index with args: {"input":"chapter 7"}
Got output: Chapter 7 discusses various aspects of error handling, such as using exceptions instead of return codes, writing try-catch-finally statements first, using unchecked exceptions, providing context with exceptions, defining exception classes based on a caller's needs, defining the normal flow, avoiding returning null, and not passing null.

Chapter 7 of the book focuses on error handling. It covers topics such as using exceptions instead of return codes, writing try-catch-finally statements first, using unchecked exceptions, providing context with exceptions, defining exception classes based on a caller's needs, defining the normal flow, avoiding returning null, and not passing null.


In [37]:
from llama_parse import LlamaParse
pdf_file_name = '/Users/doduyhiep/chainLit/data/Clean.Code.A.Handbook.of.Agile.Software.Craftsmanship.pdf'
documents = LlamaParse(api_key= llama_key,result_type="markdown").load_data(pdf_file_name)
print(documents)

Started parsing the file under job_id b3544ab5-8ba9-4568-a210-e27dbd3e16c7
.....

KeyboardInterrupt: 